In [1]:
import os
import sys
import pandas as pd
from sklearn.model_selection import train_test_split

import dataset

In [2]:
dataset_file_path = os.path.join("sensor_data.csv")
sensor_data = pd.read_csv(dataset_file_path)

y = sensor_data.gesture
X = sensor_data.drop(["gesture"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, test_size=0.1)

Approach 1: Random Forest

In [3]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [4]:
def rf_get_mae(max_leaf_nodes, train_X, test_X, train_y, test_y):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes)
    model.fit(train_X, train_y)
    pred_y = model.predict(test_X)
    mae = mean_absolute_error(test_y, pred_y)
    return mae

In [5]:
# find optimal max leaf nodes
candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500, 1000, 1500]

# iterate through all candidates and find the best size
optimal_size = 0
min_mae = sys.maxsize
for i in candidate_max_leaf_nodes:
    current_mae = rf_get_mae(i, X_train, X_test, y_train, y_test)
    if current_mae < min_mae:
        optimal_size = i
        min_mae = current_mae

In [6]:
# final model
rf_model = RandomForestRegressor(max_leaf_nodes=optimal_size)

# train model
rf_model.fit(X_train, y_train)

RandomForestRegressor(max_leaf_nodes=100)

In [7]:
# prediction
rf_pred_y = rf_model.predict(X_test)
rf_mse = mean_absolute_error(rf_pred_y, y_test)
print("MSE for Random Forest: ", rf_mse)

MSE for Random Forest:  0.06329780743342835


Test prediction

In [11]:
import numpy as np

In [15]:
test_pred_result = rf_model.predict(np.asarray([[0, 10,154, 201]]))
print(test_pred_result)

[1.]


/Users/allynbao/miniconda3/envs/tensorflow_2.9/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Approach 2: Neural Network

In [8]:
import tensorflow as tf

In [10]:
# preprocessing - normalization
normalization_layer = tf.keras.layers.experimental.preprocessing.Normalization()
normalization_layer.adapt(X_train.to_numpy())

2023-06-16 16:14:10.119014: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [22]:
# neural net model
nn_model = tf.keras.Sequential([
    normalization_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [23]:
# train model
nn_model.compile(optimizer='adam',
                 loss='mean_squared_error')

nn_model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/10
31/31 [==============================] - 0s 2ms/step - loss: 3.0271 - val_loss: 2.7017
Epoch 2/10
31/31 [==============================] - 0s 917us/step - loss: 3.0258 - val_loss: 2.7015
Epoch 3/10
31/31 [==============================] - 0s 886us/step - loss: 3.0258 - val_loss: 2.7015
Epoch 4/10
31/31 [==============================] - 0s 803us/step - loss: 3.0255 - val_loss: 2.7014
Epoch 5/10
31/31 [==============================] - 0s 832us/step - loss: 3.0255 - val_loss: 2.7014
Epoch 6/10
31/31 [==============================] - 0s 782us/step - loss: 3.0254 - val_loss: 2.7014
Epoch 7/10
31/31 [==============================] - 0s 815us/step - loss: 3.0254 - val_loss: 2.7014
Epoch 8/10
31/31 [==============================] - 0s 814us/step - loss: 3.0253 - val_loss: 2.7014
Epoch 9/10
31/31 [==============================] - 0s 825us/step - loss: 3.0253 - val_loss: 2.7014
Epoch 10/10
31/31 [==============================] - 0s 786us/step - loss: 3.0253 - val_loss: 2.7014


In [19]:
# prediction
nn_pred_y = nn_model.predict(X_test, batch_size=16)
nn_mse = mean_absolute_error(nn_pred_y, y_test)
print("MSE for Neural Net: ", nn_mse)

4/4 [==============================] - 0s 676us/step
MSE for Neural Net:  0.9444444444444444


convert model to tflite model

In [28]:
converter = tf.lite.TFLiteConverter.from_keras_model(rf_model)
tflite_model = converter.convert()

open("rf_lite_model.tflite", 'wb').write(tflite_model)

AttributeError: 'RandomForestRegressor' object has no attribute 'call'